In [1]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

In [7]:
from datasets import load_dataset
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import numpy as np
import librosa
import torch

In [24]:
# Specify the path to your audio file
audio_file_path = "audio01-park.webm"
audio_sample, sampling_rate = librosa.load(audio_file_path, sr=None)

target_sampling_rate=16000
waveform = librosa.resample(audio_sample, orig_sr=sampling_rate, target_sr=target_sampling_rate)

/tmp/ipykernel_46740/824959599.py:3: UserWarning: PySoundFile failed. Trying audioread instead.
  audio_sample, sampling_rate = librosa.load(audio_file_path, sr=None)
/home/hugo/miniconda3/envs/cp-snips/lib/python3.8/site-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


In [25]:
# Load the Whisper model in Hugging Face format:
processor = WhisperProcessor.from_pretrained("openai/whisper-tiny.en")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny.en")
model = model.to("cuda")
# t = model.cuda()

In [26]:
# Use the model and processor to transcribe the audio:
input_features = processor(
    waveform, sampling_rate=target_sampling_rate, return_tensors="pt"
).input_features

input_features = input_features.to("cuda")

In [27]:
torch.cuda.empty_cache()
predicted_ids = model.generate(input_features, num_beams=50, output_scores=True, num_return_sequences=50, return_dict_in_generate=True)

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 3.81 GiB of which 16.69 MiB is free. Including non-PyTorch memory, this process has 3.79 GiB memory in use. Of the allocated memory 3.63 GiB is allocated by PyTorch, and 87.00 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [22]:
transcriptions = processor.batch_decode(predicted_ids['sequences'], skip_special_tokens=True)

In [23]:
transcriptions

[' Take me to office 13.',
 ' Take me to office 13',
 ' Take me to office thirteen.',
 ' Take me to Office 13.',
 ' Take me to Office 13',
 ' Take me to office thirteen',
 ' Take Me to Office 13.',
 ' Take me to office 13!',
 ' Thank me to Office 13.',
 ' take me to office 13',
 ' Thank me to office 13.',
 ' Take me to office, 13.',
 ' Take Me to Office 13',
 ' Take me to office-13.',
 ' Take me to office 30.',
 ' Take me 2 of his 13.',
 ' Take me to office thirteen!',
 ' take me to office 13.',
 ' Take me to office 13,',
 ' Take me, to office 13.',
 ' Take me two of his thirteen.',
 ' Take me to Office 13!',
 ' Take me 2 of his 13th.',
 ' Thank me to Office 13',
 ' Take me to office thirty.',
 ' Take me to office 13...',
 ' Take me to Office13.',
 ' Take me to of his 13th.',
 ' Take me to office, thirteen.',
 ' Thank me, to Office 13.',
 ' Take me, to Office 13.',
 ' Thank me to office 13',
 ' Take me to office-13',
 ' Take me to of his thirteen.',
 ' Take me to of his 13.',
 ' Take m

In [15]:
def softmax(x):
    exp_x = np.exp(x - np.max(x))  # Subtracting max(x) for numerical stability
    return exp_x / np.sum(exp_x)
    
softmax(np.array(predicted_ids.sequences_scores.to("cpu")))

array([0.03242296, 0.03065933, 0.02639749, 0.02420337, 0.0222497 ,
       0.02184635, 0.02180174, 0.02168724, 0.02167558, 0.02117742,
       0.02088895, 0.02084779, 0.02076587, 0.0206741 , 0.02062286,
       0.01992741, 0.0198622 , 0.01980323, 0.01968788, 0.01963754,
       0.01949405, 0.01945613, 0.01939586, 0.01939478, 0.01931476,
       0.01912552, 0.01904396, 0.01901469, 0.01899287, 0.01873998,
       0.01861763, 0.01850006, 0.01849323, 0.01848104, 0.01847809,
       0.01836319, 0.01831727, 0.01815892, 0.01808033, 0.01803855,
       0.01791137, 0.01785957, 0.01784545, 0.01781416, 0.01779658,
       0.01776871, 0.01774761, 0.01773078, 0.01762584, 0.01756003],
      dtype=float32)